In [ ]:
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os, gc, cv2, random, warnings, math, sys, json, pprint
from glob import glob
from pylab import rcParams
from sklearn.model_selection import  GroupKFold
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam


In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

In [ ]:
data_dir = '../input/ranzcr-clip-catheter-line-classification'
os.listdir(data_dir)

In [ ]:
print('Train images: %d' %len(os.listdir(os.path.join(data_dir, "train"))))

In [ ]:
from kaggle_datasets import KaggleDatasets

data_dir ='../input/ranzcr-clip-catheter-line-classification'
path_dir = KaggleDatasets().get_gcs_path('ranzcr-clip-catheter-line-classification')

In [ ]:
train = os.path.join(data_dir, 'train.csv')
train_df = pd.read_csv(train)  
print("data_train_csv : " + str(train_df.shape[0]))


sub = os.path.join(data_dir,'sample_submission.csv')
sub_df = pd.read_csv(sub)
print("data_submission_csv : " + str(sub_df.shape[0]) )


label_cols = sub_df.columns[1:]
labels = train_df[label_cols].values

train_images = path_dir + "/train/" + train_df['StudyInstanceUID'] + '.jpg'   
test_images = path_dir + "/test/" + sub_df['StudyInstanceUID'] + '.jpg'

In [ ]:
label_cols.shape[0]

In [ ]:
BATCH_SIZE=8*REPLICAS 
STEPS_PER_EPOCH = len(train_df) * 0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train_df) * 0.2 / BATCH_SIZE
EPOCHS = 30
TARGET_SIZE = 750
N_LABELS=label_cols.shape[0]

def build_decoder(with_labels = True,
                  target_size = (TARGET_SIZE, TARGET_SIZE), 
                  ext = 'jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels = 3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels = 3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels = True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_brightness(img,0.2)
        img = tf.image.random_contrast(img,0.2,0.5)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels = None, bsize = BATCH_SIZE, cache = True,
                  decode_fn = None, augment_fn = None,
                  augment = True, repeat = True, shuffle = 1024, 
                  cache_dir = ""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls = AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls = AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:
train_img, valid_img, train_labels, valid_labels = train_test_split(train_images, labels, train_size = 0.8, random_state = 42)


In [ ]:
valid_labels.shape

In [ ]:
train_df = build_dataset(
     train_img, train_labels, bsize = BATCH_SIZE, 
     cache = True)

valid_df = build_dataset(
     valid_img, valid_labels, bsize = BATCH_SIZE, 
     repeat = False, shuffle = False, augment = False, 
     cache = True)

test_df = build_dataset(
    test_images, bsize = BATCH_SIZE, repeat = False, 
    shuffle = False, augment = False, cache = False)

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2,Xception,ResNet152,EfficientNetB7,NASNetLarge

def create_model_Xception():
    conv_base = Xception(include_top = False, weights = 'imagenet',
                          input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
    model = conv_base.output
    print(conv_base.output.shape)
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dropout(0.25)(model)
    model = layers.Dense(11, activation = "sigmoid")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = Adam(lr = 0.0001),
                   loss = "binary_crossentropy",
                   metrics = [tf.keras.metrics.AUC(multi_label = True)])
    return model

def create_model_InceptionResNetV2():
    conv_base = InceptionResNetV2(include_top = False, weights = 'imagenet',
                          input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
    model = conv_base.output
    print(conv_base.output.shape)

    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dropout(0.25)(model)
    model = layers.Dense(11, activation = "sigmoid")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = Adam(lr = 0.0001),
                   loss = "binary_crossentropy",
                   metrics = [tf.keras.metrics.AUC(multi_label = True)])
    return model

# def create_model_ResNet152():
#     conv_base = ResNet152(include_top = False, weights = 'imagenet',
#                           input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
#     model = conv_base.output
#     model = layers.GlobalAveragePooling2D()(model)
#     model = layers.Dropout(0.25)(model)
#     model = layers.Dense(11, activation = "sigmoid")(model)
#     model = models.Model(conv_base.input, model)

#     model.compile(optimizer = Adam(lr = 0.001),
#                    loss = "binary_crossentropy",
#                    metrics = [tf.keras.metrics.AUC(multi_label = True)])
#     return model

# def create_model_EfficientNetB7():
#     conv_base = EfficientNetB7(include_top = False, weights = 'imagenet',
#                           input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
#     model = conv_base.output
#     model = layers.GlobalAveragePooling2D()(model)
#     model = layers.Dropout(0.25)(model)
#     model = layers.Dense(11, activation = "sigmoid")(model)
#     model = models.Model(conv_base.input, model)

#     model.compile(optimizer = Adam(lr = 0.001),
#                    loss = "binary_crossentropy",
#                    metrics = [tf.keras.metrics.AUC(multi_label = True)])
#     return model





In [ ]:
with strategy.scope():
    model_Xception = create_model_Xception()
    model_InceptionResNetV2 = create_model_InceptionResNetV2()
    model_Xception.load_weights('../input/ranzcr-models/model_Xception_best.h5')
    model_InceptionResNetV2.load_weights('../input/ranzcr-models/model_InceptionResNetV2_best.h5')

#     model_ResNet152 = create_model_ResNet152()
#     model_EfficientNetB7 = create_model_EfficientNetB7()




In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.0001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.4, 
                              patience = 2, min_delta = 0.0001, 
                              mode = 'min', verbose = 1)




In [ ]:
model_Xception.save('./model_Xception.h5')
model_Xception_save = ModelCheckpoint('./model_Xception_best.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)


model_InceptionResNetV2.save('./model_InceptionResNetV2.h5')
model_InceptionResNetV2_save = ModelCheckpoint('./model_InceptionResNetV2_best.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)


# model_ResNet152.save('./model_ResNet152.h5')
# model_ResNet152_save = ModelCheckpoint('./model_ResNet152_best.h5', 
#                              save_best_only = True, 
#                              save_weights_only = True,
#                              monitor = 'val_loss', 
#                              mode = 'min', verbose = 1)

# model_EfficientNetB7.save('./model_EfficientNetB7.h5')
# model_EfficientNetB7_save = ModelCheckpoint('./model_EfficientNetB7_best.h5', 
#                              save_best_only = True, 
#                              save_weights_only = True,
#                              monitor = 'val_loss', 
#                              mode = 'min', verbose = 1)


In [ ]:
history_model_Xception = model_Xception.fit(train_df,
                    epochs=EPOCHS,
                    steps_per_epoch = STEPS_PER_EPOCH,
                    validation_data=valid_df,
                    validation_steps = VALIDATION_STEPS,
                    callbacks = [model_Xception_save,early_stop,reduce_lr]
                   )

history_model_InceptionResNetV2 = model_InceptionResNetV2.fit(train_df,
                    epochs=EPOCHS,
                    steps_per_epoch = STEPS_PER_EPOCH,
                    validation_data=valid_df,
                    validation_steps = VALIDATION_STEPS,
                    callbacks = [model_InceptionResNetV2_save,early_stop,reduce_lr]
                   )
# history_model_ResNet152 = model_ResNet152.fit(train_df,
#                     epochs=EPOCHS,
#                     steps_per_epoch = STEPS_PER_EPOCH,
#                     validation_data=valid_df,
#                     validation_steps = VALIDATION_STEPS,
#                     callbacks = [model_ResNet152_save,early_stop,reduce_lr]
#                    )
# history_model_EfficientNetB7 = model_EfficientNetB7.fit(train_df,
#                     epochs=EPOCHS,
#                     steps_per_epoch = STEPS_PER_EPOCH,
#                     validation_data=valid_df,
#                     validation_steps = VALIDATION_STEPS,
#                     callbacks = [model_EfficientNetB7_save,early_stop,reduce_lr]
#                    )

In [ ]:
print(history_model_Xception.history.keys())
print(history_model_InceptionResNetV2.history.keys())



In [ ]:
plt.figure()
plt.plot(history_model_Xception.history['loss'])
plt.plot(history_model_Xception.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.figure()
plt.plot(history_model_Xception.history['auc'])
plt.plot(history_model_Xception.history['val_auc'])

plt.title('Xception AUC')
plt.ylabel('AUC score')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.figure()
plt.plot(history_model_InceptionResNetV2.history['loss'])
plt.plot(history_model_InceptionResNetV2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.figure()
plt.plot(history_model_InceptionResNetV2.history['auc_1'])
plt.plot(history_model_InceptionResNetV2.history['val_auc_1'])

plt.title('InceptionResNetV2 AUC')
plt.ylabel('AUC score')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
v1 = valid_labels.reshape(66187,)
def getaccuracy(r1):
    count=0
    for i in range(len(r1)):
        if (r1[i]<0.5 and v1[i]<0.5) or (r1[i]>=0.5 and v1[i]>=0.5):
            count+=1
    return count/len(r1)

In [ ]:

result = model_Xception.predict(valid_df)


In [ ]:
print(getaccuracy(result.reshape(66187,)))

In [ ]:
result2 = model_InceptionResNetV2.predict(valid_df)

In [ ]:
print(getaccuracy(result2.reshape(66187,)))

In [ ]:
result

In [ ]:
models = [model_Xception,model_InceptionResNetV2]
yhats = [model.predict(valid_df) for model in models]



In [ ]:
yhats=np.array(yhats)
summed = np.sum(yhats, axis=0)
result=summed/2
from sklearn.metrics import accuracy_score

In [ ]:

v1

In [ ]:
print(getaccuracy(result.reshape(66187,)))

ensemble accuracy: 0.9455

In [ ]:
v1=valid_labels.reshape(66187,)

In [ ]:

from sklearn.metrics import roc_curve
fpr_ensemble, tpr_ensemble, thresholds_ensemble = roc_curve(valid_labels.reshape(66187,), result.reshape(66187,))
from sklearn.metrics import auc
auc_ensemble = auc(fpr_ensemble, tpr_ensemble)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_ensemble, tpr_ensemble, label='Ensemble (area = {:.3f})'.format(auc_ensemble))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.savefig('Ensemble')
plt.show()




In [ ]:
whats=yhats
weights=[0.1,0.3,0.5,0.7]
weightedresults=np.zeros((4,6017,11))
for w in range(len(weights)):
    for i in range(whats.shape[0]):
        for j in range(whats.shape[1]):
            for k in range(whats.shape[2]):
                if i == 0:
                    whats[i][j][k] *= weights[w]
                else:
                    whats[i][j][k] *= weights[(1-w)]
    summed = np.sum(whats, axis=0)
    weightedresults[w] = summed
    whats=yhats
    


                    
    

In [ ]:
from sklearn.metrics import roc_curve
y_pred_Xception = model_Xception.predict(valid_df).ravel()
# fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)
y_pred_Incep = model_InceptionResNetV2.predict(valid_df).ravel()

In [ ]:
y_pred_Xception.shape

In [ ]:
valid_labels.shape

In [ ]:

vl.shape

In [ ]:
fpr_Xception, tpr_Xception, thresholds_Xception = roc_curve(valid_labels.reshape(66187,), y_pred_Xception)
fpr_Incep, tpr_Incep, thresholds_Incep = roc_curve(valid_labels.reshape(66187,), y_pred_Incep)

from sklearn.metrics import auc
auc_Xception = auc(fpr_Xception, tpr_Xception)
auc_incep = auc(fpr_Incep,tpr_Incep)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_Xception, tpr_Xception, label='Xception (area = 0.976)')
plt.plot(fpr_Incep, tpr_Incep, label='InceptionResNetV2 (area = 0.973)')
plt.plot(fpr_ensemble, tpr_ensemble, label='Ensemble (area = {:.3f})'.format(auc_ensemble))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.savefig('AUCfinal')
plt.show()


In [ ]:
# result_Xception =  model_Xception.predict(test_df, verbose = 1)
# result_InceptionResNetV2 =  model_InceptionResNetV2.predict(test_df, verbose = 1)
# # result_ResNet152 =  model_ResNet152.predict(test_df, verbose = 1)
# # result_EfficientNetB7 =  model_EfficientNetB7.predict(test_df, verbose = 1)
# result = (result_Xception+result_InceptionResNetV2)/2

# sub_df[label_cols] = result
# sub_df.to_csv('submission.csv', index = False)

In [ ]:
print("done!")